## 1. Загрузка данных

In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import inv
from sklearn.metrics import mean_squared_error

Нам дана информация о поле, возрасте, зарплате и количестве членов семьи клиентов страховой компании, а также целевой признак - количество страховых выплат клиентам за последние 5 лет. Изучим данные подробнее.

In [2]:
data = pd.read_csv('datasets/insurance.csv')
display(data.head())
print('Частота страховых выплат\n', data['Страховые выплаты'].value_counts())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Частота страховых выплат
 0    4436
1     423
2     115
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64


Страховых выплат у клиентов в выборке от 0 до 5, при этом наибольшее число клиентов не обращается за страховыми выплатами. Но есть один клиент, обратившийся за выплатой 5 раз, из любопытсва посмотрим, что с ним не так.

In [3]:
data[data['Страховые выплаты'] == 5]

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
228,1,65.0,39700.0,1,5


Возможно, просто невезучий или невезучая. Переименуем колонки, чтобы было удобнее к ним обращаться.

In [4]:
data = data.rename(columns={
    'Пол': 'gender',
    'Возраст': 'age',
    'Зарплата': 'salary',
    'Члены семьи': 'family_members',
    'Страховые выплаты': 'insurance_payments'
})

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gender              5000 non-null   int64  
 1   age                 5000 non-null   float64
 2   salary              5000 non-null   float64
 3   family_members      5000 non-null   int64  
 4   insurance_payments  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропусков в данных нет, проверим есть ли дубликаты.

In [6]:
data.duplicated().sum()

153

In [7]:
data.drop_duplicates(inplace=True)

Удалили дубли, посмотрим, как распределены признаки.

In [8]:
data.describe()

,gender,age,salary,family_members,insurance_payments
count,4847.000000,4847.000000,4847.000000,4847.000000,4847.000000
mean,0.498453,31.023932,39895.811842,1.203425,0.152259
std,0.500049,8.487995,9972.953985,1.098664,0.468934
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33200.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


В данных примерно поровну мужчин и женщин, более половины из них в возрасте до 30 с зарплатой до 41 000 с 1 членом семьи. Более 85% без страховых выплат за последние 5 лет. Данные изучили, переходим к их преобразованию.

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Ответим на вопрос: Изменится ли качество линейной регрессии, если умножить признаки на обратимую матрицу?

**Ответ:** При умножении признаков на обратимую матрицу качество линейной регрессии не изменится. Матрица должна быть размерности p×p, где p - количество признаков в исходной задаче, в нашем случае - 4 (пол, возраст, зп и количество членов семьи).

**Обоснование:** Предположим, что признаки умножают на обратимую матрицу P размерности 4×4, тогда формула обучения примет вид:

$$
w1 = ((XP)^T (XP))^{-1} (XP)^T y
$$

Раскроем скобки, используя свойства матриц: $(AB)^T = B^T A^T,$ если произведение матриц определено.

В нашем случае при умножении X размерности 5000×4 на P размерности 4×4, получим матрицу XP размерности 5000×4 (то есть умножение определено):

$$
w1 = (P^T X^T X P)^{-1} P^T X^T y
$$

Учитывая ассоциативность умножения матриц, $(AB)C = A(BC)$, можем расставить скобки следующим образом:

$$
w1 = (P^T (X^T X P))^{-1} P^T X^T y
$$

Матрица: $X^T X P$ - это квадратная матрица размерности 4×4. Обратная матрица существует только для квадратных матриц, значит мы можем применить свойство обратной матрицы произведения: $(AB)^{-1} = B^{-1} A^{-1}$, получим:

$$
w1 = (X^T X P)^{-1} (P^T)^{-1} P^T X^T y
$$

Матрица: $(P^T)^{-1} P^T$ - это единичная матрица - ее можно опустить. Матрица: $X^T X$ - это квадратная матрица размерности 4×4, применим повторно свойство обратной матрицы произведения:

$$
w1 = P^{-1} (X^T X)^{-1} X^T y
$$

Таким образом, финальная формула обучения выглядит так:

$$
w1 = P^{-1}w
$$

Формула для расчета предсказаний:

$$
a1 = (XP) w1
$$

Подставим в нее финальную формулу обучения:

$$
a1 = X P P^{-1} w
$$

$$
a1 = X w
$$

Получается, что a1=a, другими словами, предсказания не зависят от того, что мы умножаем признаки на обратимую матрицу.

## 3. Алгоритм преобразования

**Алгоритм**

Для того чтобы защитить данные клиентов страховой компании, будем преобразовывать данные согласно следующему алгоритму:

1. Генерируем случайную обратимую квадратную матрицу P размерности p×p, где p - количество признаков в выборке без учета целевого признака.

2. Умножаем признаки на P - получаем преобразованные данные, по которым сложно восстановить исходные признаки, не зная матрицы P.

3. Передаем преобразованные данные для обучения и предсказания линейной регрессии.

**Обоснование**

При преобразовании согласно выбранному алгоритму качество линейной регрессии не ухудшится: как было доказано выше, умножение признаков на квадратную обратимую матрицу не ухудшает качества линейной регрессии и никак не влияет на предсказания.

## 4. Проверка алгоритма

Создадим класс для обучения линейной регресии и предсказания с помощью нее.

In [9]:
class LinearRegression:
    def fit(self, features_train, target_train):
        X = np.concatenate((np.ones((features_train.shape[0], 1)), 
                            features_train), axis=1)
        y = target_train
        w = inv(X.T @ X) @ X.T @ y
        self.w = w[1:]
        self.w0 = w[0]
        
    def predict(self, features_test):
        return features_test @ self.w + self.w0

Разделим данные на признаки и целевой признак. Делить данные на обучающую и тестовую выборки не нужно, так как мы не будем подбирать гиперпараметры, а просто проверим насколько хорошо обучается регрессия на одной выборке.

In [10]:
features = data.drop('insurance_payments', axis=1)
target = data['insurance_payments']

Обучим линейную регрессию и сделаем предсказания на незашифрованных данных.

In [11]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(mean_squared_error(target, predictions))

0.12527263826681603


Качество линейной регрессии оценили с помощью метрики MSE = 0.125, у нас не стоит задача минимизировать значение метрики - нужно только показать, что она не отличается до и после преобразования. Создадим функцию для шифрования данных.

In [12]:
def encrypted(features):
    P = np.random.normal(size=(features.shape[1], features.shape[1]))
    while True:
        try:
            inv(P)
        except:
            print("irreversible matrix")
            P = np.random.normal(size=(features.shape[1], features.shape[1]))
        else:
            break
    return features @ P

Необратимые матрицы встречаются редко. Если сгенерировать случайную матрицу функцией numpy.random.normal(), вероятность получить необратимую матрицу близка к нулю. Но на всякий случай мы все же проверим матрицу на обратимость с помощью функции numpy.linalg.inv(): если матрица необратима, будет обнаружена ошибка, и мы сгенерируем новую случайную матрицу.

In [13]:
features_encrypted = encrypted(features)
model.fit(features_encrypted, target)
predictions = model.predict(features_encrypted)
print(mean_squared_error(target, predictions))

0.12527263826684087


Значение метрики MSE = 0.125, таким образом, видим, что качество линейной регрессии не отличается до и после преобразования (умножения признаков на обратимую матрицу).

Цель достигнута: нам удалось защитить данные клиентов страховой компании. Был разработан алгоритм обработки данных, не ухудшающий качество моделей машинного обучения при преобразовании данных и по преобразованным данным сложно восстановить персональную информацию.